In [1]:
import os
import argparse
import math
import random
import tqdm
import numpy as np
import pandas as pd
from sklearn import preprocessing

from script import dataloader, utility, earlystopping
from model import models

import matplotlib.pyplot as plt
import seaborn as sns
import transbigdata as tbd

import networkx as nx

from typing import List

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils

from torch_geometric.data import Data

# @TODO

# station数据预处理函数
def station2staion_no_line(station):
    station_no_line = pd.DataFrame()
    for i in range(len(station['站点名'].unique())):
        spec_time_place_station = station[station['站点名'] == station['站点名'].unique()[i]]
        # spec_time_place_station = spec_time_place_station[['开始时间','线路名','站点名','进站量','出站量']]
        spec_time_place_station = spec_time_place_station.fillna(0)
        spec_time_place_station = spec_time_place_station.groupby(['日期','开始时间','结束时间']).agg({
            '站点名': 'first',  # 假设每个时间段内站点名是相同的，取第一个即可
            '进站量': 'sum',
            '出站量': 'sum'
        })
        spec_time_place_station = spec_time_place_station.reset_index()
        station_no_line = pd.concat([station_no_line, spec_time_place_station]).reset_index(drop=True)
    return station_no_line


# prepare data
line,stop = tbd.getbusdata('北京',['1号线', '2号线', '5号线', '6号线', '7号线', '8号线', '9&房山线','房山线', '10号线',
       '13号线', '15号线', '昌平线', '首都机场线'])
line = line[line['line']!='大兴机场大巴首都机场线'].reset_index(drop=True)
stop = stop[stop['line']!='大兴机场大巴首都机场线'].reset_index(drop=True)
line = line[line['line']!='首都机场大巴昌平线'].reset_index(drop=True)
stop = stop[stop['line']!='首都机场大巴昌平线'].reset_index(drop=True)
stop.loc[stop['stationnames'] == '清河站', 'stationnames'] = '清河'

line_data = {'线路名': [], '站点名': [], '顺序': []}
for i in range(len(stop)):
    线路名 = line_data['线路名']
    线路名.append(stop.loc[i, 'line'])
    站点名 = line_data['站点名']
    站点名.append(stop.loc[i, 'stationnames'])
    顺序 = line_data['顺序']
    顺序.append(stop.loc[i, 'id'])
    line_data.update({'线路名':线路名, '站点名':站点名, '顺序':顺序})
# 示例数据
# line_data = {
#     '线路名': ['1号线', '1号线', '1号线', '2号线', '2号线'],
#     '站点名': ['站点A', '站点B', '站点C', '站点B', '站点D'],
#     '顺序': [1, 2, 3, 1, 2]
# }


stop_data = {'站点名':list(stop['stationnames'].unique())}
# 示例数据
# stop_data = {
#     '站点名': ['站点A', '站点B', '站点C', '站点D'],
#     '其他信息': ['infoA', 'infoB', 'infoC', 'infoD']
# }

line_for_adj = pd.DataFrame(line_data)
stop_for_adj = pd.DataFrame(stop_data)

# 构建地铁网络图
G = nx.Graph()

# 添加节点
for station_for_adj in stop_for_adj['站点名']:
    G.add_node(station_for_adj)

# 添加边（根据站点顺序）
for line_name, group in line_for_adj.groupby('线路名'):
    sorted_stations = group.sort_values('顺序')['站点名'].tolist()
    for i in range(len(sorted_stations) - 1):
        G.add_edge(sorted_stations[i], sorted_stations[i + 1])

# 提取邻接矩阵
adj_matrix = nx.adjacency_matrix(G).todense()

# 读取csv
path = 'CTS-2024-dataset/'
station_csv_file = ['station_20230519.csv',
                    'station_20230520.csv',
                    'station_20230521.csv',
                    'station_20230522.csv',
                    'station_20230523.csv',
                    'station_20230524.csv',
                    'station_20230525.csv',]
station_input_csv_select = range(len(station_csv_file))
station_csv_sel_list = [path + station_csv_file[i] for i in station_input_csv_select]
# stationList = utility.cstRawCsvData([path + station_csv_file[i] for i in station_input_csv_select])
station = utility.get_station_for_adj(stop_for_adj, station_csv_sel_list)
station = station2staion_no_line(station) # 合并分布在不同线路上的同一站点数据



Obtaining city id: 北京success
Get bus data: 1号线
地铁1号线八通线(环球度假区-古城) success
地铁1号线八通线(古城-环球度假区) success
地铁1号线支线(八角游乐园-青龙湖东) success
地铁1号线支线(青龙湖东-八角游乐园) success
Get bus data: 2号线
地铁2号线外环(西直门-西直门) success
地铁2号线内环(积水潭-积水潭) success
地铁6号线二期(潞城-东小营南) success
地铁6号线二期(东小营南-潞城) success
Get bus data: 5号线
地铁5号线(宋家庄-天通苑北) success
地铁5号线(天通苑北-宋家庄) success
Get bus data: 6号线
地铁6号线(金安桥-潞城) success
地铁6号线(潞城-金安桥) success
Get bus data: 7号线
地铁7号线(北京西站-环球度假区) success
地铁7号线(环球度假区-北京西站) success
Get bus data: 8号线
地铁8号线(朱辛庄-瀛海) success
地铁8号线(瀛海-朱辛庄) success
Get bus data: 9&房山线
地铁9号线(国家图书馆-郭公庄) success
地铁9号线(郭公庄-国家图书馆) success
Get bus data: 房山线
地铁房山线(东管头南-阎村东) success
地铁房山线(阎村东-东管头南) success
大兴机场大巴房山线(房山区交通局-大兴机场) success
大兴机场大巴房山线(大兴机场-房山区交通局) success
Get bus data: 10号线
地铁10号线外环(车道沟-车道沟) success
地铁10号线内环(巴沟-巴沟) success
Get bus data: 13号线
地铁13号线(东直门-西直门) success
地铁13号线(西直门-东直门) success
地铁13A号线(天通苑东-车公庄) success
地铁13A号线(车公庄-天通苑东) success
Get bus data: 15号线
地铁15号线(清华东路西口-俸伯) success
地铁15号线(俸伯-清华东路西口) success
Get bus 

/home/africar/CTS-transmit/.venv/lib/python3.12/site-packages/transbigdata/crawler.py:284: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  data['geometry'] = lines
/home/africar/CTS-transmit/script/utility.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_df = pd.concat([output_df, new_row], ignore_index=True)


tmp_rem_df.__len__ is  6480


/home/africar/CTS-transmit/script/utility.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_df = pd.concat([output_df, new_row], ignore_index=True)


tmp_rem_df.__len__ is  6480


...

tmp_rem_df.__len__ is  6480


/home/africar/CTS-transmit/script/utility.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_df = pd.concat([output_df, new_row], ignore_index=True)


tmp_rem_df.__len__ is  6480


/home/africar/CTS-transmit/script/utility.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_df = pd.concat([output_df, new_row], ignore_index=True)


tmp_rem_df.__len__ is  6480
output_df.__len__ is  106848


In [2]:
# 数据预加载
from datetime import datetime

sta_info_for_adj = pd.DataFrame(columns=['station_name', 'abs_date', 'time_point', 'day_of_week', 'is_event', 'in_flow', 'out_flow'])
for i in range(len(station)):
    station_name = station.loc[i, '站点名']
    precise_time = datetime.strptime(station.loc[i, '开始时间'], '%Y/%m/%d %H:%M:%S')

    time_diff = precise_time - datetime.strptime('2023/5/19 00:00:00', '%Y/%m/%d %H:%M:%S')
    abs_date = time_diff.days + 1 # 从1开始编号，五月19是第一天

    time_point = (precise_time.hour * 60 + precise_time.minute) / 30

    day_of_week = precise_time.weekday() # Monday == 0 ... Sunday == 6

    event_condition = ((precise_time.day == 26 or precise_time.day == 27) and precise_time.month == 5) or ((precise_time.day == 25) and precise_time.month == 8)
    is_event = 1 if event_condition else 0

    in_flow = float(station.loc[i, '进站量'])
    out_flow = float(station.loc[i, '出站量'])

    sta_info_for_adj.loc[i] = [station_name, abs_date, time_point, day_of_week, is_event, in_flow, out_flow]


In [3]:
def get_station_mask(stop_for_adj: pd.DataFrame, csv_list: list[str]) -> torch.tensor:
    output_mask = torch.ones(len(stop_for_adj)).tolist()
    categories_list = list(stop_for_adj['站点名'])
    rem_stop_list = utility.get_rem_sta_list(stop_for_adj, csv_list)
    for stop_i in range(len(categories_list)):
        if categories_list[stop_i] in rem_stop_list:
            output_mask[stop_i] = False

    for i in range(len(output_mask)):
        if output_mask[i]:
            output_mask[i] = True

    return torch.tensor(output_mask)

station_mask = get_station_mask(stop_for_adj, station_csv_sel_list)
station_mask

the length of remain_staion_list is  135


tensor([ True, False, False, False, False,  True, False,  True,  True, False,
        False,  True, False,  True,  True, False,  True, False,  True,  True,
         True,  True, False,  True,  True,  True, False, False,  True,  True,
         True,  True,  True, False, False, False, False, False, False, False,
        False, False, False, False,  True,  True, False, False, False,  True,
         True,  True,  True, False,  True,  True,  True,  True,  True, False,
         True, False, False,  True,  True, False, False, False,  True, False,
         True,  True,  True,  True, False,  True,  True,  True,  True, False,
         True,  True, False, False, False, False,  True, False, False, False,
        False,  True, False, False,  True,  True,  True, False, False,  True,
         True, False,  True,  True, False, False, False,  True,  True, False,
         True,  True,  True, False,  True,  True, False,  True,  True,  True,
         True,  True, False, False,  True, False, False, False, 

In [4]:
from sklearn.preprocessing import StandardScaler

categories_list = list(stop_for_adj['站点名'])
scaler = StandardScaler()
my_x = []
my_y = []
for i in sta_info_for_adj.groupby(["abs_date","time_point"]):
    temp = i[1]
    temp['station_name'] = pd.Categorical(temp['station_name'], categories=categories_list, ordered=True)
    temp = temp.sort_values(by=['station_name']).reset_index(drop=True)#特定日期特定时间的所有站点已知数据
    features = temp[['abs_date','time_point','day_of_week','is_event']].values
    labels = temp[['in_flow','out_flow']].values
    features = scaler.fit_transform(features)
    my_x.append(features)
    my_y.append(labels)

my_x = np.array(my_x)
my_y = np.array(my_y)



# 准备模型输入与loss标杆
# 找到所有非零元素的索引
adj_matrix = np.array(adj_matrix)
row, col = np.nonzero(adj_matrix)

# 将这些索引转换为 PyTorch 张量
edge_index = torch.tensor(np.array([row, col]), dtype=torch.long)
data_input: List[Data] = [Data(x=torch.tensor(my_x[epoch], dtype=torch.float),
                               edge_index=edge_index,
                               y=torch.tensor(my_y[epoch], dtype=torch.float),
                               node_mask=station_mask)
                          for epoch in range(len(my_x))]

In [5]:
station_mask

tensor([ True, False, False, False, False,  True, False,  True,  True, False,
        False,  True, False,  True,  True, False,  True, False,  True,  True,
         True,  True, False,  True,  True,  True, False, False,  True,  True,
         True,  True,  True, False, False, False, False, False, False, False,
        False, False, False, False,  True,  True, False, False, False,  True,
         True,  True,  True, False,  True,  True,  True,  True,  True, False,
         True, False, False,  True,  True, False, False, False,  True, False,
         True,  True,  True,  True, False,  True,  True,  True,  True, False,
         True,  True, False, False, False, False,  True, False, False, False,
        False,  True, False, False,  True,  True,  True, False, False,  True,
         True, False,  True,  True, False, False, False,  True,  True, False,
         True,  True,  True, False,  True,  True, False,  True,  True,  True,
         True,  True, False, False,  True, False, False, False, 

In [6]:
# prepare model parameters
num_features = 4
num_classes = 2
model = models.GCN(num_features, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()


In [11]:
# start training
duptimes = 400 # 每个数据输入进模型训练的次数
total_input_sz = len(data_input)
test_set_sz = 48 # 预测集大小
train_set_sz = total_input_sz - test_set_sz
model.train()
for dup_i in range(duptimes):
    optimizer.zero_grad()  # 清除梯度
    for epoch in range(train_set_sz):
        out_y = model(data_input[epoch])  # 前向传播
        out_y1_for_loss = out_y[:, 0][station_mask]
        out_y2_for_loss = out_y[:, 1][station_mask]
        true_y1_for_loss = data_input[epoch].y[:, 0][station_mask]
        true_y2_for_loss = data_input[epoch].y[:, 1][station_mask]
        loss1 = criterion(out_y1_for_loss, true_y1_for_loss)  # 计算损失
        loss2 = criterion(out_y2_for_loss, true_y2_for_loss)
    loss = loss1 + loss2  # 加权求和
    if dup_i % (duptimes / 20) == 0:
        print('dup_i=', dup_i, '\t\tloss=', loss.item())
    loss.backward()  # 反向传播
    optimizer.step()  # 更新参数



dup_i= 0 		loss= 2394.588623046875
dup_i= 20 		loss= 2394.5859375
dup_i= 40 		loss= 2394.585205078125
dup_i= 60 		loss= 2394.584716796875
dup_i= 80 		loss= 2394.584716796875
dup_i= 100 		loss= 2394.5849609375
dup_i= 120 		loss= 2394.584716796875
dup_i= 140 		loss= 2394.584716796875
dup_i= 160 		loss= 2394.584716796875
dup_i= 180 		loss= 2394.584716796875
dup_i= 200 		loss= 2394.584716796875
dup_i= 220 		loss= 2394.584716796875
dup_i= 240 		loss= 2394.584716796875
dup_i= 260 		loss= 2394.584716796875
dup_i= 280 		loss= 2394.584716796875
dup_i= 300 		loss= 2394.584716796875
dup_i= 320 		loss= 2394.584716796875
dup_i= 340 		loss= 2394.584716796875
dup_i= 360 		loss= 2394.584716796875
dup_i= 380 		loss= 2394.584716796875


In [18]:
# make prediction
model.eval()
predict_data: List[Data] = data_input[train_set_sz:]

predict_in_flow = []
predict_out_flow = []
for piece in predict_data:
    pre_out = model(piece)
    predict_in_flow.append(pre_out[:, 0][station_mask])
    predict_out_flow.append(pre_out[:, 1][station_mask])

In [19]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 将预测值和真实值转换为 numpy 数组以便使用 sklearn 库计算评估指标
true_in_flow = np.array([data.y[:, 0][station_mask].detach().numpy() for data in predict_data])
true_out_flow = np.array([data.y[:, 1][station_mask].detach().numpy() for data in predict_data])
predicted_in_flow = np.array([out.detach().numpy() for out in predict_in_flow])
predicted_out_flow = np.array([out.detach().numpy() for out in predict_out_flow])

# 计算 MAE 和 RMSE
mae_in = mean_absolute_error(true_in_flow, predicted_in_flow)
rmse_in = np.sqrt(mean_squared_error(true_in_flow, predicted_in_flow))

mae_out = mean_absolute_error(true_out_flow, predicted_out_flow)
rmse_out = np.sqrt(mean_squared_error(true_out_flow, predicted_out_flow))

print(f"MAE for In-Flow: {mae_in}, RMSE for In-Flow: {rmse_in}")
print(f"MAE for Out-Flow: {mae_out}, RMSE for Out-Flow: {rmse_out}")

MAE for In-Flow: 359.4963684082031, RMSE for In-Flow: 728.1917724609375
MAE for Out-Flow: 357.58306884765625, RMSE for Out-Flow: 769.3988037109375
